## Image Data

In [20]:
import os
import torch
import imageio.v2 as iio

In [9]:
img_arr = iio.imread('data/bobby.jpg')

# (height, width, channels)
img_arr.shape

(720, 1280, 3)

In [10]:
# Numpy arrays perfect for converting to tensors
type(img_arr)

numpy.ndarray

In [13]:
# PyTorch needs shape (channels, height, width)
img = torch.from_numpy(img_arr)

# permute() simply requires the desired ordering of the dimensions
# and works inplace (also changing img will change out)
out = img.permute(2, 0, 1)
out.shape

torch.Size([3, 720, 1280])

In [14]:
# To build up a stack of tensors, preallocate a tensor of the desired size
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

In [16]:
# Load data into the prepared tensor
data_dir = 'data/ch4/image-cats/'
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == '.png']
for i, filename in enumerate(filenames):
    img_arr = iio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3]   # Sometimes, images have an alpha channel
    batch[i] = img_t    # Add to batch

In [18]:
# Normalize: NNs generally work best with inputs between 0 and 1 or -1 and 1

# One possibility: Simply scale down
batch = batch.float()
batch /= 255

# Or: standardize each channel
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std

## Medical Data

Of a CT scan

In [23]:
dir_path = 'data/ch4/volumetric-dicom/2-LUNG 3.0  B70f-04083'
vol_arr = iio.volread(dir_path, 'DICOM')
vol_arr.shape, type(vol_arr)

Reading DICOM (examining files): 99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 99/99  (100.0%)


((99, 512, 512), imageio.core.util.Array)

In [26]:
# Add channel dimension
vol = torch.from_numpy(vol_arr).float()
vol = torch.unsqueeze(vol, 0)
vol.shape

torch.Size([1, 99, 512, 512])

## Tabular Data

In [60]:
import csv
import numpy as np

wine_path = 'data/ch4/tabular-wine/winequality-white.csv'
wine_np = np.loadtxt(wine_path, dtype=np.float32, delimiter=';', skiprows=1)
wine_np.shape

(4898, 12)

In [61]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))
col_list

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [62]:
wine_t = torch.from_numpy(wine_np)
wine_t.shape, wine_t.dtype

(torch.Size([4898, 12]), torch.float32)

In [63]:
# Split into inputs and targets
data = wine_t[:, :-1]
data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [64]:
target = wine_t[:, -1]
target, target.shape

(tensor([6., 6., 6.,  ..., 6., 7., 6.]), torch.Size([4898]))

In [65]:
target

tensor([6., 6., 6.,  ..., 6., 7., 6.])

In [66]:
# We can treat targets like numbers for a regression
target = target.long()

In [67]:
# Or as labels
# Onehot encoding is only required for inputs, targets can remain simply classes
target_onehot = torch.zeros(target.shape[0], 10)

# Arguments:
#   Dimension along which the following two arguments are specified
#   Column tensor indicating the indices of the elements to scatter
#   The value to use
# The output and input tensors have to have same shape, so we use unsqueeze
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [68]:
# Get mean of each column
data_mean = torch.mean(data, dim=0)
data_std = torch.std(data, dim=0)

# Normalize/Standardize
data_norm = (data - data_mean) / data_std

In [69]:
# Find bad wines
bad_indexes = target <= 3
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [70]:
bad_data = data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

In [71]:
mid_data = data[(target > 3) & (target < 7)]
good_data = data[(target >= 7)]

In [72]:
bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

In [73]:
for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [74]:
total_sulfur_threshold = 141.83
total_sulfur_data = data[:, 6]
predicted_indices = torch.lt(total_sulfur_data, total_sulfur_threshold)
predicted_indices.shape, predicted_indices.dtype, predicted_indices.sum()

(torch.Size([4898]), torch.bool, tensor(2727))

In [76]:
actual_indices = target > 5
actual_indices.shape, actual_indices.dtype, actual_indices.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

In [77]:
n_matches = torch.sum(actual_indices & predicted_indices).item()
n_predicted = torch.sum(predicted_indices).item()
n_actual = torch.sum(actual_indices).item()
n_matches, n_matches / n_predicted, n_matches / n_actual

(2018, 0.74000733406674, 0.6193984039287906)

## Time-Series Data

In [81]:
bikes_numpy = np.loadtxt(
    'data/ch4/bike-sharing-dataset/hour-fixed.csv',
    dtype=np.float32,
    delimiter=',',
    skiprows=1,
    converters={1: lambda x: float(x[8:10])}
)
bikes = torch.from_numpy(bikes_numpy)
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))